# 1. Introdução

Os objetivos deste notebook auxiliar são listados a seguir: 
- Acessar de forma programática os dados de operações de financiamento não automáticas do BNDES
- Explorar as categorias de setor e subsetor do bndes
- Padronizar o formato das variáveis e seus valores
- Deflacionar os dados de desembolso, usando o índice IGPM com base no mês de dezembro de 2010 (mesma base dos dados do PIB).
- Agregar a base com os dados do número de empregos e emissão de CO2 formatadas nos notebooks: auxCAGED e auxSEEG.
- Agregar a base com dados do PIB.
- Agregar a base com dados de IDH.

## 1.1 - Dados de operações de financiamento não automáticas

### Overview do dataset
Operações contratadas diretamente com o BNDES e operações contratadas via agente financeiro (indiretas) que foram revisadas por analistas do BNDES. O critério principal para seja feita análise manual de operações indiretas é o valor.

### Contratos e subcréditos
Cada contrato pode ter um ou mais subcréditos, sendo que cada subscrédito pode ser caracterizado por condições financeiras distintas.

O somatório dos valores dos subcréditos equivale ao valor total do contrato.

Cada linha corresponde a um subcrédito diferente.

### Valor contratado e desembolsado em R$
O valor contratado das operações do BNDES pode ser corrigido por algum indexador (US$, IPCA, TJLP etc), caso haja previsão no contrato.

O valor contratado em R$ representa o valor convertido com base na data do contrato e no indexador.

O valor desembolsado representa o valor liberado convertido com base na data do desembolso e no indexador. Desta forma, caso haja variação positiva do indexador, entre a data do contrato e a data do desembolso, o valor desembolsado pode ser maior que o valor contratado.

## 1.2 - Dados do SEEG
O Sistema de Estimativas de Emissões e Remoções de Gases de Efeito Estufa(SEEG), desenvolvido pelo Observatório do Clima, disponibiliza estimativas anuais das emissões de gases de efeito estufa(GEE) no Brasil. São considerados os gases de efeito estufa(como CO2, CH4, N2O, os HFCs, e apresentados como CO2e) e avaliados todos os cinco setores que são fontes de emissões – Agropecuária, Energia, Mudanças de Uso da Terra, Processos Industriais e Resíduos, em cidades e estados do Brasil.

As estimativas são feitas de acordo com as diretrizes do Painel Intergovernamental sobre Mudanças Climáticas(IPCC), com base na metodologia dos Inventários Brasileiros de Emissões e Remoções Antrópicas de Gases do Efeito Estufa, elaborado pelo Ministério da Ciência, Tecnologia e Inovação(MCTI), e em dados obtidos junto a relatórios governamentais, institutos, centros de pesquisa, entidades setoriais e organizações não governamentais

Essas informações podem ser encontradas no site do SEEG (http://seeg.eco.br/), onde também é possível visualizar e fazer download das estimativas, assim como ver detalhes sobre a metodologia e a qualidade dos dados. Link para artigo com a metodologia do SEEG, publicado na NATURE, em 2018: https://www.nature.com/articles/sdata201845.


Foi usado o gás co2e(t) GWP-AR5 como medida de emissão, pois o CO2e (CO2 equivalente) converte todas as outras medidas e gases em CO2. O GWP é a métrica de conversão mais comum, com base na energia e potencial de aquecimento. O AR5 é o mais recente.

O agrupamento para o setor Agrícola e Industrial foi definido a partir da segmentação indicada pelo SEEG (imagem a baixo).
Dessa forma, a classificação final de grande setor foi a seguinte:

- Agropecuária = Agropecuária (nível 1) + Mudança de Uso da Terra e Florestas (nível 1)
- Indústria = Processos Industriais (nivel 1)

Nota-se que uma pequena fração da emissão agropecuária vem de resíduos, e cerca de 20% da emissão industrial vem de energia. 
Para agregar estes dados, é necessário iterar por inúmeras tabelas do SEEG que iriam consumir grande parte do tempo disponível para o concurso, por isso é importante ressaltar que as emissões não estár 100% ajustadas. Essa sugestão fica para futuros trabalhos, que podem inclusive segmentar a emissão por nível 3 e conseguir maior granularidade na análise.

Adicionalmente, analisaremos apenas emissao de co2, e não remoção.

![Categorias SEEG](./img/categorias_seeg.jpg)

## 1.3 - Dados do CAGED
Foram recolhidos dados do número de empregos formais da plataforma CAGED do governo. Os dados estão segmentados por ano, setor, estado e tamanho da empresa.

A classificação do tamanho da empresa foi realizada através do padrão adotado no CAGED, que segue o seguinte formato:

Micro: com até 19 empregados.
Pequena: de 20 a 99 empregados.
Média: 100 a 499 empregados.
Grande: mais de 500 empregados.

Os dados estão disponíveis para consulta no seguinte endereço: bi.mte.gov.br/bgcaged

## 1.4 - Dados do PIB
Foram levantados dados do produto interno bruto do setor Industrial e Agrícola, para os anos de 2002 a 2017 de todos os estados brasileiros.

Os dados estão em escala de R$1000,00, com a PIB real em valores de 2010 pelo deflator implícito do PIB nacional.
Esta escala será revertida para o valor em unidade de real.

Os dados do pib estão disponíveis em: http://www.ipeadata.gov.br/Default.aspx

## 1.5 - Dados de IDH
Neste trabalho serão usados dados apenas do IDHM, pois fazem parte do escopo que busca-se analisar.
Os dados de IDH estão disponíveis para os estados brasileiros nos anos de 2010, 2016 e 2017.

Dados disponíveis em: http://www.atlasbrasil.org.br/ranking

# 2. Carregando dados e dependências

## 2.1 Importando dependências

Importando as bibliotecas necessárias para as etapas inicias de carregamento de dados, exploração e transformação.

In [1]:
## Load dependencies.
import pandas as pd
import json
import requests
import numpy as np
from global_functions import strip_upper
from global_functions import map_values_caged
from global_functions import map_values_general
from global_functions import quote
from global_functions import melt_pib
from global_functions import search_info

## 2.1 Carregando dados através da API CKAN.

A base de dados carregada aqui é referente as operações de financiamento não automáticas dos dados abertos do BNDES.

In [2]:
## Define resource identifier and server to create our URL string for an API request.
res_id = '332d446e-d340-46ef-af64-ee6f36e7bd50' ## Operações de financiamento não automáticas.
server = 'https://dadosabertos.bndes.gov.br'

## Define query limit.
limit = 20000

## Define filters.
## This project works with only those sectors.
filters = quote({'setor_bndes': ['INFRAESTRUTURA', 'INDUSTRIA', 'AGROPECUÁRIA']})
                 
## Define fields to query.
fields = 'cliente,cnpj,uf, numero_do_contrato, data_da_contratacao,valor_contratado_reais, \
            valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro, \
            juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,produto, \
            instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes, \
            porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato'

## Create query url.
query = (f"{server}/api/3/action/datastore_search?resource_id={res_id}"
         f"&filters={filters}&fields={fields}&limit={limit}")

print('Url para consulta:', query)

Url para consulta: https://dadosabertos.bndes.gov.br/api/3/action/datastore_search?resource_id=332d446e-d340-46ef-af64-ee6f36e7bd50&filters=%7B%22setor_bndes%22%3A%20%5B%22INFRAESTRUTURA%22%2C%20%22INDUSTRIA%22%2C%20%22AGROPECU%5Cu00c1RIA%22%5D%7D&fields=cliente,cnpj,uf, numero_do_contrato, data_da_contratacao,valor_contratado_reais,             valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,             juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,produto,             instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes,             porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato&limit=20000


In [3]:
## Request dataset.
try:
    data = requests.get(query)
    if data.json()['success'] == True:
        df_fin = pd.DataFrame(data.json()['result']['records'])

    else:
        print(data.json()['error'])
except:
    data.raise_for_status()


# 3. Data wrangling - Transformando e mapeando os dados para análise.


## 3.1 Explorando o dataset 'Operações não automáticas'

In [4]:
## Overview of the dataset.
df_fin.head()

,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,produto,instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato
0,ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...,61.695.227/0001-93,SP,2204531,2002-02-04T00:00:00,1.820297e+08,1.820297e+08,RECURSOS LIVRES - TESOURO,TAXA FIXA,1.0,...,BNDES FINEM,OUTROS,NÃO,AREA DE ENERGIA,INFRAESTRUTURA,ENERGIA ELÉTRICA,GRANDE,PRIVADA,REAL,LIQUIDADO
1,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,1235151,2002-01-10T00:00:00,5.000000e+05,5.339624e+05,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,TJLP,1.0,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
2,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,1235151,2002-01-10T00:00:00,5.714000e+06,5.340896e+06,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,US$ / CESTA,4.5,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
3,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,1235151,2002-01-10T00:00:00,2.968000e+07,2.999406e+07,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,TJLP,4.5,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
4,USINA CAETE S A,12.282.034/0001-03,AL,1252231,2002-01-11T00:00:00,6.323444e+06,6.408000e+06,RECURSOS LIVRES - PRÓPRIOS / RECURSOS VINCULAD...,TJLP,3.5,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INFRAESTRUTURA,ENERGIA ELÉTRICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO


In [36]:
df_fin.loc[0]

cliente                         ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...
cnpj                                                           61.695.227/0001-93
uf                                                                             SP
numero_do_contrato                                                        2204531
data_da_contratacao                                           2002-02-04T00:00:00
valor_contratado_reais                                                182029685.4
valor_desembolsado_reais                                              182029685.4
fonte_de_recurso_desembolsos                            RECURSOS LIVRES - TESOURO
custo_financeiro                                                        TAXA FIXA
juros                                                                         1.0
prazo_carencia_meses                                                           63
prazo_amortizacao_meses                                                         8
modalidade_de_ap

In [14]:
## Veryifing duplicated records
df_fin.duplicated().sum()

541

In [15]:
## Dropping duplicates
df_fin = df_fin.drop_duplicates(subset=None, keep='first', inplace = False, ignore_index = True)


In [16]:
## Veryifing duplicated records
df_fin.duplicated().sum()

0

In [8]:
## Dataset dimensions
df_fin.shape

(14955, 23)

In [9]:
## Veryfing NA's
df_fin.isna().sum()

cliente                         0
cnpj                            0
uf                              0
numero_do_contrato              0
data_da_contratacao             0
valor_contratado_reais          0
valor_desembolsado_reais        0
fonte_de_recurso_desembolsos    0
custo_financeiro                0
juros                           0
prazo_carencia_meses            0
prazo_amortizacao_meses         0
modalidade_de_apoio             0
produto                         0
instrumento_financeiro          0
inovacao                        0
area_operacional                0
setor_bndes                     0
subsetor_bndes                  0
porte_do_cliente                0
natureza_do_cliente             0
tipo_de_garantia                0
situacao_do_contrato            0
dtype: int64

In [10]:
## Printing unique bndes sector
print(df_fin['setor_bndes'].unique())

## Printing unique bndes sub sector
print(df_fin['subsetor_bndes'].unique())


['INFRAESTRUTURA' 'INDUSTRIA' 'AGROPECUÁRIA']
['ENERGIA ELÉTRICA' 'MECÂNICA' 'MATERIAL DE TRANSPORTE' 'AGROPECUÁRIA'
 'OUTROS TRANSPORTES' 'TÊXTIL E VESTUÁRIO' 'OUTRAS' 'ALIMENTO E BEBIDA'
 'METALURGIA E PRODUTOS' 'CONSTRUÇÃO' 'QUÍMICA E PETROQUÍMICA'
 'TRANSPORTE RODOVIÁRIO' 'SERV. UTILIDADE PÚBLICA' 'ATV. AUX. TRANSPORTES'
 'EXTRATIVA' 'TRANSPORTE FERROVIÁRIO' 'CELULOSE E PAPEL'
 'TELECOMUNICAÇÕES']


In [17]:
## Subsectors of AGROPECUÁRIA
df_fin.loc[df_fin['setor_bndes'] == 'AGROPECUÁRIA']['subsetor_bndes'].unique()

array(['AGROPECUÁRIA'], dtype=object)

In [18]:
## Subsectors of INFRAESTRUTURA
df_fin.loc[df_fin['setor_bndes'] == 'INFRAESTRUTURA']['subsetor_bndes'].unique()


array(['ENERGIA ELÉTRICA', 'OUTROS TRANSPORTES', 'CONSTRUÇÃO',
       'TRANSPORTE RODOVIÁRIO', 'SERV. UTILIDADE PÚBLICA',
       'ATV. AUX. TRANSPORTES', 'TRANSPORTE FERROVIÁRIO',
       'TELECOMUNICAÇÕES'], dtype=object)

In [13]:
## Subsectors of INDUSTRIA
df_fin.loc[df_fin['setor_bndes'] == 'INDUSTRIA']['subsetor_bndes'].unique()


array(['MECÂNICA', 'MATERIAL DE TRANSPORTE', 'TÊXTIL E VESTUÁRIO',
       'OUTRAS', 'ALIMENTO E BEBIDA', 'METALURGIA E PRODUTOS',
       'QUÍMICA E PETROQUÍMICA', 'EXTRATIVA', 'CELULOSE E PAPEL'],
      dtype=object)

In [12]:
## Type of support
df_fin['modalidade_de_apoio'].unique()

array(['REEMBOLSÁVEL', 'NÃO REEMBOLSÁVEL'], dtype=object)

In [13]:
## Print number of different supports
print(len(df_fin.loc[df_fin['modalidade_de_apoio'] == 'REEMBOLSÁVEL']))
print(len(df_fin.loc[df_fin['modalidade_de_apoio'] == 'NÃO REEMBOLSÁVEL']))

14705
250


Financimanentos não reembolsáveis são doações que o BNDES faz para cultura/esportes e outros incentivos sociais. Por terem apenas 250 observações e serem desembolsos com características diferentes das que procuramos analisar neste trabalho, iremos retirar estes dados da base.

In [19]:
## Dropping type 'NÃO REEMBOLSÁVEL'.
df_fin = df_fin.loc[df_fin['modalidade_de_apoio'] == 'REEMBOLSÁVEL']
len(df_fin)

14705

In [20]:
## Understading contract categories.
print(df_fin['situacao_do_contrato'].unique())
print(len(df_fin.loc[df_fin['situacao_do_contrato'] == '-']))


['LIQUIDADO' 'ATIVO' '-']
189


In [21]:
## We lack information to know what '-' means in the contracts.
## Since they can have different interpretations, they are dropped here.
df_fin = df_fin.loc[df_fin['situacao_do_contrato'] != '-']
len(df_fin)


14516

## 3.2 Padronizando a formatação dos dados

In [22]:
## Adjusting index.
df_fin.index = pd.RangeIndex(len(df_fin.index))

In [23]:
# Indexing a new column with year-month format
df_fin['ano'] = pd.to_datetime(df_fin['data_da_contratacao']).dt.to_period('m')


In [24]:
## Strip setor,subsetor BNDES and porte_do_cliente from special characters
df_fin['setor_bndes'] = strip_upper(df_fin['setor_bndes'].tolist(),'')
df_fin['subsetor_bndes'] = strip_upper(df_fin['subsetor_bndes'].tolist(), '')
df_fin['porte_do_cliente'] = strip_upper(df_fin['porte_do_cliente'].tolist(), '')
## Printing unique bndes sector
print(df_fin['setor_bndes'].unique())

## Printing unique bndes sub sector
print(df_fin['subsetor_bndes'].unique())

['INFRAESTRUTURA' 'INDUSTRIA' 'AGROPECUARIA']
['ENERGIA ELETRICA' 'MECANICA' 'MATERIAL DE TRANSPORTE' 'AGROPECUARIA'
 'OUTROS TRANSPORTES' 'TEXTIL E VESTUARIO' 'OUTRAS' 'ALIMENTO E BEBIDA'
 'METALURGIA E PRODUTOS' 'CONSTRUCAO' 'QUIMICA E PETROQUIMICA'
 'TRANSPORTE RODOVIARIO' 'SERV. UTILIDADE PUBLICA' 'ATV. AUX. TRANSPORTES'
 'EXTRATIVA' 'TRANSPORTE FERROVIARIO' 'CELULOSE E PAPEL'
 'TELECOMUNICACOES']


## 3.3 - Deflacionando valores de desembolso
O índice usado para deflacionar a série é o IGP-M calculado pela FGV. A série história do índice pode ser acessada em: https://sindusconpr.com.br/igp-m-fgv-309-p

A data usada como base é de dezembro de 2010.

In [25]:
## Load IGP-M
igpm = pd.read_csv('bases-auxiliares/igpm.csv')

In [26]:
igpm

,ano,igpm
0,1994-08-01,"100,000"
1,1994-09-01,"101,751"
2,1994-10-01,"103,602"
3,1994-11-01,"106,553"
4,1994-12-01,"107,450"
...,...,...
319,2021-03-01,"1.011,948"
320,2021-04-01,"1.027,211"
321,2021-05-01,"1.069,289"
322,2021-06-01,"1.075,733"


In [27]:
## Convert data to year-month format
igpm['ano'] = pd.to_datetime(igpm['ano']).dt.to_period('m')

In [28]:
## Convert numeric format to EU standard
igpm['igpm'] = igpm['igpm'].str.replace('\.', '')
igpm['igpm'] = igpm['igpm'].str.replace(',', '.')


/tmp/ipykernel_602/521327833.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  igpm['igpm'] = igpm['igpm'].str.replace('\.', '')


In [29]:
## Transform IGP-M to a dict
igpm_dict = dict(zip(igpm['ano'].dt.strftime('%Y-%m'), igpm['igpm']))

In [30]:
## append igpm information to our main dataset
df_fin['igpm'] = df_fin['ano'].dt.strftime('%Y-%m').map(igpm_dict)

In [32]:
## set 2010-12 as our base date
base_igpm = float(igpm_dict['2010-12'])
base_igpm

450.301

In [34]:
## Deflacionando valor_contratado
for i in range(len(df_fin)):
    df_fin.loc[:i, 'valor_contratado_ajustado'] = round((base_igpm / df_fin.loc[:i, 'igpm'].astype(float)) *
          (df_fin.loc[:i, 'valor_contratado_reais'].astype(float)))


In [27]:
## Deflacionando valor_desembolsado
for i in range(len(df_fin)):
    df_fin.loc[:i, 'valor_desembolsado_ajustado'] = round((base_igpm / df_fin.loc[:i, 'igpm'].astype(float)) *
         (df_fin.loc[:i, 'valor_desembolsado_reais'].astype(float)))


In [28]:
## Checking NA.
df_fin.isna().sum()

cliente                         0
cnpj                            0
uf                              0
numero_do_contrato              0
data_da_contratacao             0
valor_contratado_reais          0
valor_desembolsado_reais        0
fonte_de_recurso_desembolsos    0
custo_financeiro                0
juros                           0
prazo_carencia_meses            0
prazo_amortizacao_meses         0
modalidade_de_apoio             0
produto                         0
instrumento_financeiro          0
inovacao                        0
area_operacional                0
setor_bndes                     0
subsetor_bndes                  0
porte_do_cliente                0
natureza_do_cliente             0
tipo_de_garantia                0
situacao_do_contrato            0
ano                             0
igpm                            0
valor_contratado_ajustado       0
valor_desembolsado_ajustado     0
dtype: int64

In [29]:
## Checking 0's and negative values
df_fin.loc[df_fin['valor_contratado_ajustado'] <= 0]

,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato,ano,igpm,valor_contratado_ajustado,valor_desembolsado_ajustado


In [30]:
df_fin.loc[df_fin['valor_desembolsado_ajustado'] <= 0]

,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato,ano,igpm,valor_contratado_ajustado,valor_desembolsado_ajustado
23,COMPANHIA BRASILEIRA DE OFFSHORE,13.534.284/0001-48,RJ,1244531,2002-02-04T00:00:00,4004991.76,0.0,RECURSOS VINCULADOS - TESOURO,US$ / CESTA,5.50,...,INFRAESTRUTURA,OUTROS TRANSPORTES,MEDIA,PRIVADA,"REAL / PESSOAL / OUTRA, DE NATUREZA ESPECÍFICA...",ATIVO,2002-02,217.074,8308005.0,0.0
169,VICUNHA SIDERURGIA S/A.,02.871.007/0001-04,RJ,1230311,2002-03-25T00:00:00,40977230.05,0.0,-,TJLP,3.75,...,INDUSTRIA,METALURGIA E PRODUTOS,GRANDE,PRIVADA,DEFINIDA PELO AGENTE FINANCEIRO,ATIVO,2002-03,217.276,84924647.0,0.0
170,VICUNHA SIDERURGIA S/A.,02.871.007/0001-04,RJ,1230312,2002-03-25T00:00:00,45657080.84,0.0,-,TJLP,3.75,...,INDUSTRIA,METALURGIA E PRODUTOS,GRANDE,PRIVADA,DEFINIDA PELO AGENTE FINANCEIRO,ATIVO,2002-03,217.276,94623562.0,0.0
173,VICUNHA SIDERURGIA S/A.,02.871.007/0001-04,RJ,1230313,2002-03-25T00:00:00,9930415.09,0.0,-,TJLP,3.75,...,INDUSTRIA,METALURGIA E PRODUTOS,GRANDE,PRIVADA,DEFINIDA PELO AGENTE FINANCEIRO,ATIVO,2002-03,217.276,20580625.0,0.0
238,EMPRESA AMAZONENSE DE TRANSMISSAO DE ENERGIA S/A,04.416.935/0001-04,PA,2219241,2002-06-11T00:00:00,12530960.00,0.0,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,US$ / CESTA,5.00,...,INFRAESTRUTURA,ENERGIA ELETRICA,GRANDE,PRIVADA,REAL,LIQUIDADO,2002-06,223.688,25225778.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14511,MUELLER FOGOES LTDA.,04.565.361/0001-36,SC,19400011,2021-05-27T00:00:00,4479999.98,0.0,-,TLP,1.39,...,INDUSTRIA,MECANICA,GRANDE,PRIVADA,PESSOAL,ATIVO,2021-05,1069.289,1886626.0,0.0
14512,HERCULES MOTORES ELETRICOS LTDA,07.442.711/0001-65,SC,19400011,2021-05-27T00:00:00,736000.00,0.0,-,TLP,1.39,...,INDUSTRIA,MECANICA,GRANDE,PRIVADA,PESSOAL,ATIVO,2021-05,1069.289,309946.0,0.0
14513,MUELLER FOGOES LTDA.,04.565.361/0001-36,SC,19400011,2021-05-27T00:00:00,88000.00,0.0,-,TLP,1.39,...,INDUSTRIA,MECANICA,GRANDE,PRIVADA,PESSOAL,ATIVO,2021-05,1069.289,37059.0,0.0
14514,MUELLER FOGOES LTDA.,04.565.361/0001-36,SC,19400011,2021-05-27T00:00:00,104564.00,0.0,-,TLP,1.39,...,INDUSTRIA,MECANICA,GRANDE,PRIVADA,PESSOAL,ATIVO,2021-05,1069.289,44034.0,0.0


In [31]:
## Removing those observations, which are outside the scope of this project.
df_fin = df_fin.loc[(df_fin['valor_desembolsado_ajustado']!=0)]

In [32]:
## Exporting dataset
df_fin.to_csv('bases-ajustadas/operacoes-ajustado.csv',
                index=False, header=True)

# 4 - Agrupando dados para receber bases externas

## 4.1 - Agrupamento para os dados CAGED - UF/SETOR/ANO/PORTE_DO_CLIENTE


Os dados do CAGED tem um nível de granularidade maior do que os dados do SEEG e do PIB, devido a presença da segmentação por porte do cliente. Devido a isso, o primeiro agrupamento será para este formato.

In [33]:
## Loading auxiliar datasets.
caged = pd.read_csv('bases-auxiliares/CAGED-ajustado.csv')

In [34]:
## Setting dataset for CAGED data.
df_caged = df_fin

In [35]:
## Drop columns with higher segmentation levels.
drop_cols = ['cliente', 'cnpj', 'ano', 'fonte_de_recurso_desembolsos', 'numero_do_contrato',
             'custo_financeiro', 'modalidade_de_apoio', 'produto', 'instrumento_financeiro',
             'inovacao', 'area_operacional', 'subsetor_bndes', 'natureza_do_cliente', 'tipo_de_garantia',
             'situacao_do_contrato', 'igpm']
df_caged.drop(drop_cols, axis=1, inplace=True)


In [36]:
## Indexing a new column with year format.
df_caged['ano'] = pd.to_datetime(df_caged['data_da_contratacao'])
df_caged['ano'] = pd.DatetimeIndex(df_caged['ano']).year


In [37]:
## Group dataset with mean values and by uf, setor_bndes, porte_do_cliente and ano.
df_caged = df_caged.groupby(['uf', 'setor_bndes', 'porte_do_cliente', 'ano'], as_index=False).agg(
    {'valor_contratado_reais': ['sum'], 'valor_desembolsado_reais': ['sum'],
     'juros': ['mean'], 'prazo_carencia_meses': ['mean'], 'prazo_amortizacao_meses': ['mean'],
     'valor_contratado_ajustado': ['sum'], 'valor_desembolsado_ajustado': ['sum']})


In [38]:
## Melt multi index from pandas.
df_caged.columns = list(map(''.join, df_caged.columns.values))

In [39]:
## Rename columns.
df_caged.columns = ['uf', 'setor_bndes', 'porte_do_cliente', 'ano', 'valor_contratado_reais',
                      'valor_desembolsado_reais', 'media_juros', 'media_prazo_carencia_meses',
                      'media_prazo_amortizacao_meses', 'valor_contratado_ajustado', 'valor_desembolsado_ajustado']


In [40]:
## Round numeric values.
toRound = ['valor_contratado_reais', 'valor_desembolsado_reais',
            'valor_contratado_ajustado', 'valor_desembolsado_ajustado',
            'media_prazo_carencia_meses', 'media_prazo_amortizacao_meses']
df_caged[toRound] = df_caged[toRound].round(1)
df_caged['media_juros'] = df_caged['media_juros'].round(2)


In [41]:
# Re-order columns.
df_caged = df_caged[['uf', 'setor_bndes', 'porte_do_cliente', 'ano', 'valor_contratado_reais',
                        'valor_desembolsado_reais', 'valor_contratado_ajustado', 'valor_desembolsado_ajustado',
                        'media_juros', 'media_prazo_carencia_meses', 'media_prazo_amortizacao_meses']]

In [42]:
## Append number of employees from CAGED dataset, segmented by company_size/sector/uf/ano.
df_caged = map_values_caged(df_caged, caged, 'setor_bndes', 'num_empregados')

In [43]:
## veryfing random observation.
print(caged.loc[(caged['uf'] == 'SP') & (caged['porte_do_cliente'] == 'GRANDE') & 
      (caged['setor'] == 'INDUSTRIA')])


       ano  uf      setor porte_do_cliente  num_empregados
4393  2019  SP  INDUSTRIA           GRANDE          741404
4420  2018  SP  INDUSTRIA           GRANDE          734995
4447  2017  SP  INDUSTRIA           GRANDE          737504
4474  2016  SP  INDUSTRIA           GRANDE          749834
4501  2015  SP  INDUSTRIA           GRANDE          809950
4528  2014  SP  INDUSTRIA           GRANDE          899721
4555  2013  SP  INDUSTRIA           GRANDE          941342
4582  2012  SP  INDUSTRIA           GRANDE          933411
4609  2011  SP  INDUSTRIA           GRANDE          951255
4636  2010  SP  INDUSTRIA           GRANDE          920209
4663  2009  SP  INDUSTRIA           GRANDE          826559
4690  2008  SP  INDUSTRIA           GRANDE          843542
4717  2007  SP  INDUSTRIA           GRANDE          807752
4744  2006  SP  INDUSTRIA           GRANDE          707430
4771  2005  SP  INDUSTRIA           GRANDE          635279
4798  2004  SP  INDUSTRIA           GRANDE          6215

In [44]:
df_caged.loc[(df_caged['uf'] == 'SP') & (df_caged['porte_do_cliente'] == 'GRANDE') &
      (df_caged['setor_bndes'] == 'INDUSTRIA')]

,uf,setor_bndes,porte_do_cliente,ano,valor_contratado_reais,valor_desembolsado_reais,valor_contratado_ajustado,valor_desembolsado_ajustado,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,num_empregados
879,SP,INDUSTRIA,GRANDE,2002,7.031570e+08,7.340150e+08,1.281616e+09,1.339494e+09,3.28,19.4,55.9,505442.0
880,SP,INDUSTRIA,GRANDE,2003,3.156121e+08,3.141715e+08,4.908004e+08,4.886598e+08,3.64,16.1,62.8,525774.0
881,SP,INDUSTRIA,GRANDE,2004,2.766436e+08,2.771103e+08,3.891016e+08,3.897936e+08,4.38,15.0,57.5,621570.0
882,SP,INDUSTRIA,GRANDE,2005,2.549802e+09,2.557720e+09,3.424499e+09,3.435183e+09,5.11,21.7,31.6,635279.0
883,SP,INDUSTRIA,GRANDE,2006,2.656786e+09,2.503691e+09,3.482421e+09,3.281751e+09,2.63,21.4,61.5,707430.0
884,SP,INDUSTRIA,GRANDE,2007,3.189347e+09,3.130468e+09,3.993045e+09,3.920485e+09,3.20,19.1,66.4,807752.0
885,SP,INDUSTRIA,GRANDE,2008,4.442376e+09,3.885050e+09,5.050306e+09,4.408068e+09,2.83,18.3,73.6,843542.0
886,SP,INDUSTRIA,GRANDE,2009,3.385919e+09,3.318741e+09,3.756893e+09,3.682427e+09,3.09,15.3,63.9,826559.0
887,SP,INDUSTRIA,GRANDE,2010,2.555657e+09,2.540039e+09,2.724112e+09,2.708024e+09,3.13,17.1,54.7,920209.0
888,SP,INDUSTRIA,GRANDE,2011,4.639901e+09,4.488574e+09,4.498715e+09,4.350856e+09,3.50,20.6,76.1,951255.0


In [45]:
## Checking 0's and negative values.
df_caged.loc[df_caged['num_empregados'] <= 0]

,uf,setor_bndes,porte_do_cliente,ano,valor_contratado_reais,valor_desembolsado_reais,valor_contratado_ajustado,valor_desembolsado_ajustado,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,num_empregados
0,AC,INDUSTRIA,GRANDE,2002,605515.2,612000.0,1195596.0,1208401.0,4.00,18.0,66.0,0.0
1,AC,INDUSTRIA,GRANDE,2005,25059451.9,25712810.3,33504343.0,34377879.0,3.50,18.0,102.0,0.0
2,AC,INDUSTRIA,GRANDE,2008,55073532.0,51807339.7,64184654.0,60378117.0,2.00,30.0,90.0,0.0
5,AL,AGROPECUARIA,GRANDE,2014,12350000.0,12350000.0,9962536.0,9962536.0,2.00,36.0,84.0,0.0
953,TO,AGROPECUARIA,GRANDE,2011,11494000.0,11494000.0,11156409.0,11156409.0,2.45,84.0,84.0,0.0


In [46]:
## Checking 0's and negative values.
df_caged.loc[df_caged['valor_contratado_ajustado'] <= 0]

,uf,setor_bndes,porte_do_cliente,ano,valor_contratado_reais,valor_desembolsado_reais,valor_contratado_ajustado,valor_desembolsado_ajustado,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,num_empregados


In [47]:
## Checking 0's and negative values.
df_caged.loc[df_caged['valor_desembolsado_ajustado'] <= 0]


,uf,setor_bndes,porte_do_cliente,ano,valor_contratado_reais,valor_desembolsado_reais,valor_contratado_ajustado,valor_desembolsado_ajustado,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,num_empregados


In [49]:
## Setting 0 to na
df_caged['num_empregados'].replace({'0': np.nan, 0: np.nan}, inplace=True)

In [50]:
## Exporting integrated dataset
df_caged.to_csv('bases-ajustadas/operacoes-uf-setor-ano-porte.csv',
                    index=False, header=True)

## 4.2 - Agrupamento para os dados do PIB/SEEG/CAGED_por_estado - UF/SETOR/ANO

- **É necessário recuperar o dataset após o passo 3.3 para rodar esta parte.**

### 4.2.1 - Agrupamento para os dados de PIB

In [51]:
## Loading auxiliar datasets.
pib_ind = pd.read_csv('bases-auxiliares/PIB-IND.csv')
pib_agro = pd.read_csv('bases-auxiliares/PIB-agro.csv')

In [52]:
## Setting dataset for PIB data.
df_pib = pd.read_csv('bases-ajustadas/operacoes-ajustado.csv')

In [53]:
## Drop columns with higher segmentation levels.
drop_cols = ['cliente', 'cnpj', 'ano', 'fonte_de_recurso_desembolsos', 'numero_do_contrato', 'igpm',
             'custo_financeiro', 'modalidade_de_apoio', 'produto', 'instrumento_financeiro',
             'inovacao', 'area_operacional', 'subsetor_bndes', 'porte_do_cliente', 'natureza_do_cliente', 'tipo_de_garantia',
             'situacao_do_contrato']
df_pib.drop(drop_cols, axis=1, inplace=True)


In [54]:
# Indexing a new column with year format.
df_pib['ano'] = pd.to_datetime(df_pib['data_da_contratacao'])
df_pib['ano'] = pd.DatetimeIndex(df_pib['ano']).year

In [55]:
## Group dataset with mean values and by uf, setor_bndes, ano.
df_pib = df_pib.groupby(['uf', 'setor_bndes', 'ano'], as_index=False).agg(
    {'valor_contratado_reais': ['sum'], 'valor_desembolsado_reais': ['sum'],
     'juros': ['mean'], 'prazo_carencia_meses': ['mean'], 'prazo_amortizacao_meses': ['mean'],
     'valor_contratado_ajustado': ['sum'], 'valor_desembolsado_ajustado': ['sum']})


In [56]:
## Melt multi index from pandas.
df_pib.columns = list(map(''.join, df_pib.columns.values))

In [57]:
## Rename columns.
df_pib.columns = ['uf', 'setor_bndes', 'ano', 'valor_contratado_reais',
                    'valor_desembolsado_reais', 'media_juros', 'media_prazo_carencia_meses',
                    'media_prazo_amortizacao_meses', 'valor_contratado_ajustado', 'valor_desembolsado_ajustado']


In [58]:
# Round numeric values.
toRound = ['valor_contratado_reais', 'valor_desembolsado_reais',
           'valor_contratado_ajustado', 'valor_desembolsado_ajustado',
           'media_prazo_carencia_meses', 'media_prazo_amortizacao_meses']
df_pib[toRound] = df_pib[toRound].round(1)
df_pib['media_juros'] = df_pib['media_juros'].round(2)


In [59]:
df_pib.head()

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado
0,AC,INDUSTRIA,2002,605515.2,612000.0,4.0,18.0,66.0,1195596.0,1208401.0
1,AC,INDUSTRIA,2005,25059451.9,25712810.3,3.5,18.0,102.0,33504343.0,34377879.0
2,AC,INDUSTRIA,2008,55073532.0,51807339.7,2.0,30.0,90.0,64184654.0,60378117.0
3,AC,INFRAESTRUTURA,2007,32048964.0,32110602.6,2.0,24.0,96.0,40890361.0,40969004.0
4,AC,INFRAESTRUTURA,2008,60759000.0,60936725.7,2.0,30.0,162.0,70810700.0,71017828.0


In [60]:
## Set columns to drop from PIB datasets
col_drops = ['Código', 'Estado']

## Change PIB datasets structure to match our project standards
pib_agro = melt_pib(pib_agro, col_drops, 'AGROPECUARIA')
pib_ind = melt_pib(pib_ind, col_drops, 'INDUSTRIA')

In [61]:
## Append PIB datasets
pib = pd.concat([pib_agro, pib_ind], axis=0, ignore_index=True)

In [62]:
print(pib.shape)
print(pib.head())
print(pib['setor'].unique())

(864, 4)
   uf   ano         setor           pib
0  AC  2002  AGROPECUARIA  5.401684e+05
1  AL  2002  AGROPECUARIA  4.700566e+06
2  AM  2002  AGROPECUARIA  2.354165e+06
3  AP  2002  AGROPECUARIA  7.825019e+04
4  BA  2002  AGROPECUARIA  1.256136e+07
['AGROPECUARIA' 'INDUSTRIA']


In [63]:
## Set format to numeric
pib['ano'] = pd.to_numeric(pib['ano'])

In [64]:
## Change scale to unit R$ and round
pib['pib'] = round(pib['pib'] * 1000)
pib

,uf,ano,setor,pib
0,AC,2002,AGROPECUARIA,5.401684e+08
1,AL,2002,AGROPECUARIA,4.700566e+09
2,AM,2002,AGROPECUARIA,2.354165e+09
3,AP,2002,AGROPECUARIA,7.825018e+07
4,BA,2002,AGROPECUARIA,1.256136e+10
...,...,...,...,...
859,RS,2017,INDUSTRIA,5.022485e+10
860,SC,2017,INDUSTRIA,3.868059e+10
861,SE,2017,INDUSTRIA,4.262866e+09
862,SP,2017,INDUSTRIA,2.318155e+11


In [67]:
## Looking for 0's and negatives.
pib.loc[pib['pib'] <= 0]

,uf,ano,setor,pib


In [19]:
## Export integrated pib dataset.
pib.to_csv('bases-auxiliares/pib-integrado.csv', index=False, header=True)

In [69]:
## Append PIB information, segmented by sector/uf/ano.
df_pib = map_values_general(df_pib, pib, 'setor_bndes', 'pib')

In [71]:
## Check for 0's and NA.
df_pib.loc[df_pib['valor_contratado_ajustado'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib


In [72]:
## Check for 0's and NA.
df_pib.loc[df_pib['valor_desembolsado_ajustado'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib


In [75]:
df = df_pib

### 4.2.2 - Agrupamento para os dados do SEEG

In [73]:
## Loading auxiliar datasets.
seeg = pd.read_csv('bases-auxiliares/SEEG-emissao-ajustado.csv')

In [76]:
## Append SEEG data.
df = map_values_general(df, seeg, 'setor_bndes', 'emissao_co2')

In [77]:
## Checking NA.
df['emissao_co2'].isna().sum()

415

In [78]:
## Checking random observation.
search_info(seeg, 'setor', 'AGROPECUARIA', 'MG', 2012)

,setor,uf,ano,emissao_co2
715,AGROPECUARIA,MG,2012,91583113.4


In [79]:
search_info(df, 'setor_bndes', 'AGROPECUARIA', 'MG', 2012)

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2
262,MG,AGROPECUARIA,2012,25056000.0,25056000.0,3.0,18.0,54.0,22262535.0,22262535.0,2.185845e+10,91583113.4


In [80]:
## Cheking for 0's and negatives
df.loc[df['emissao_co2'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2
0,AC,INDUSTRIA,2002,605515.2,612000.0,4.00,18.0,66.0,1195596.0,1208401.0,674487984.0,0.0
1,AC,INDUSTRIA,2005,25059451.9,25712810.3,3.50,18.0,102.0,33504343.0,34377879.0,598244130.0,0.0
2,AC,INDUSTRIA,2008,55073532.0,51807339.7,2.00,30.0,90.0,64184654.0,60378117.0,681922808.0,0.0
37,AP,INDUSTRIA,2007,431535057.7,425270038.7,4.23,30.0,66.0,536801874.0,529008593.0,624305588.0,0.0


In [81]:
# Check for 0's and negatives.
df.loc[df['valor_contratado_ajustado'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2


In [82]:
# Check for 0's and Negatives.
df.loc[df['valor_desembolsado_ajustado'] <= 0]


,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2


In [83]:
## Replacing 0 to NA.
df['emissao_co2'].replace({'0':np.nan, 0:np.nan}, inplace=True)

### 4.2.3 - Agrupamento para os dados do caged agregados por estado

In [84]:
## Loading auxiliar datasets.
caged = pd.read_csv('bases-auxiliares/CAGED-ajustado.csv')

In [85]:
## Drop columns with higher segmentation levels.
caged = caged.drop('porte_do_cliente', axis = 1)

In [86]:
## Group by ano, uf, setor.
caged = caged.groupby(['ano', 'uf', 'setor'], as_index=False).agg({'num_empregados': ['sum']})

In [87]:
## Melt multi index from pandas.
caged.columns = list(map(''.join, caged.columns.values))
caged

,ano,uf,setor,num_empregadossum
0,2002,AC,AGROPECUARIA,1665
1,2002,AC,CONSTRUCAO CIVIL,2882
2,2002,AC,INDUSTRIA,4177
3,2002,AL,AGROPECUARIA,16960
4,2002,AL,CONSTRUCAO CIVIL,8840
...,...,...,...,...
1453,2019,SP,CONSTRUCAO CIVIL,542457
1454,2019,SP,INDUSTRIA,2485984
1455,2019,TO,AGROPECUARIA,22232
1456,2019,TO,CONSTRUCAO CIVIL,7673


In [88]:
## Rename column.
caged.columns = ['ano', 'uf', 'setor', 'num_empregados']

In [89]:
## Append to our dataset.
# Append PIB information, segmented by sector/uf/ano.
df = map_values_general(df, caged, 'setor_bndes', 'num_empregados')

In [90]:
## Checking NA
df['num_empregados'].isna().sum()

407

In [91]:
## Checking random observation
search_info(caged, 'setor', 'INDUSTRIA', 'PR', 2014)


,ano,uf,setor,num_empregados
1025,2014,PR,INDUSTRIA,736369


In [92]:
search_info(df, 'setor_bndes', 'INDUSTRIA', 'PR', 2014)

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados
493,PR,INDUSTRIA,2014,4.320116e+09,4.353542e+09,3.58,25.3,79.4,3.499717e+09,3.526877e+09,5.588553e+10,3297438.8,736369.0


In [93]:
## Cheking for 0's and negatives
df.loc[df['emissao_co2'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados


In [94]:
# Cheking for 0's and negatives
df.loc[df['pib'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados


In [95]:
## Cheking for 0's and negatives.
df.loc[df['num_empregados'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados


In [96]:
# Check for 0's and NA.
df.loc[df['valor_contratado_ajustado'] <= 0]

,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados


In [97]:
# Check for 0's and NA.
df.loc[df['valor_desembolsado_ajustado'] <= 0]


,uf,setor_bndes,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados


In [98]:
## Exporting integrated dataset.
df.to_csv('bases-ajustadas/operacoes-uf-setor-ano.csv',
                index=False, header=True)

## 4.3 - Agrupamento para os dados de IDH - UF/ANO

In [99]:
## Loading auxiliar datasets.
idhm = pd.read_csv('bases-auxiliares/idhm.csv')

In [100]:
## Set format to numeric.
idhm['ano'] = pd.to_numeric(idhm['ano'])

In [101]:
## Grouping df to setor-ano.
df = df.groupby(['uf', 'ano'], as_index=False).agg(
    {'valor_contratado_reais': ['sum'], 'valor_desembolsado_reais': ['sum'], 'media_juros': ['mean'],
    'media_prazo_carencia_meses': ['mean'], 'media_prazo_amortizacao_meses': ['mean'],
    'valor_contratado_ajustado': ['sum'], 'valor_desembolsado_ajustado': ['sum'],
    'pib': ['sum'], 'emissao_co2': ['sum'], 'num_empregados': ['sum']})

In [102]:
## Melt multi index from pandas.
df.columns = list(map(''.join, df.columns.values))

In [103]:
## Rename columns.
df.columns = ['uf', 'ano', 'valor_contratado_reais',
                  'valor_desembolsado_reais', 'media_juros', 'media_prazo_carencia_meses',
                  'media_prazo_amortizacao_meses', 'valor_contratado_ajustado', 'valor_desembolsado_ajustado',
                  'pib', 'emissao_co2', 'num_empregados']

In [104]:
# Round numeric values.
toRound = ['valor_contratado_reais', 'valor_desembolsado_reais',
           'valor_contratado_ajustado', 'valor_desembolsado_ajustado',
           'media_prazo_carencia_meses', 'media_prazo_amortizacao_meses', 'pib',
           'emissao_co2']
df[toRound] = df[toRound].round(1)
df['media_juros'] = df['media_juros'].round(2)

In [105]:
## Append IDH information, segmented by uf/ano.
for i in range(len(idhm)):
    index = np.where(df['ano'].eq(idhm.loc[i, 'ano']) & df['uf'].eq(
        idhm.loc[i, 'uf']))

    df.loc[index[0], 'idhm'] = idhm.loc[i, 'idhm']

In [106]:
## Replacing 0's
df['pib'].replace({'0': np.nan, 0: np.nan}, inplace=True)
df['emissao_co2'].replace({'0': np.nan, 0: np.nan}, inplace=True)
df['num_empregados'].replace({'0': np.nan, 0: np.nan}, inplace=True)


In [107]:
# Cheking for 0's
df['idhm']= df['idhm'].astype(float)
df.loc[df['idhm'] == 0]

,uf,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados,idhm


In [108]:
# veryfing random observation.
idhm.loc[26:26, :]

,idhm,ano,uf
26,0.631,2010,AL


In [109]:
df.loc[df['uf'] == 'AL']

,uf,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados,idhm
4,AL,2002,51475000.0,52744954.6,2.90,30.0,120.0,106844365.0,109480353.0,NaN,NaN,NaN,NaN
5,AL,2003,91383653.3,98025890.9,1.00,10.0,24.0,141298258.0,151568548.0,NaN,NaN,NaN,NaN
6,AL,2005,12216000.0,11922045.2,2.75,9.0,72.0,16484794.0,16088120.0,NaN,NaN,NaN,NaN
7,AL,2010,527074755.5,518202362.0,2.76,15.6,68.2,527149922.0,518274785.0,4.708285e+09,242707.0,110487.0,0.631
8,AL,2011,85272847.0,85609704.5,2.43,21.0,78.0,82618512.0,82942843.0,6.010282e+09,254230.2,112755.0,NaN
9,AL,2013,310544000.0,310503000.0,2.58,16.5,78.5,271373073.0,271337245.0,4.713670e+09,294581.9,98978.0,NaN
10,AL,2014,45582000.0,40354096.8,2.96,26.2,87.8,36770231.0,32552969.0,7.430471e+09,4396421.2,100107.0,NaN
11,AL,2015,25678000.0,24444242.6,4.44,12.0,72.0,18739068.0,17838707.0,4.409744e+09,210353.4,86048.0,NaN
12,AL,2019,284845600.0,200000000.0,2.21,18.0,220.0,168968820.0,118638883.0,NaN,NaN,NaN,NaN


In [110]:
## Final overview.
df

,uf,ano,valor_contratado_reais,valor_desembolsado_reais,media_juros,media_prazo_carencia_meses,media_prazo_amortizacao_meses,valor_contratado_ajustado,valor_desembolsado_ajustado,pib,emissao_co2,num_empregados,idhm
0,AC,2002,605515.2,612000.0,4.00,18.0,66.0,1195596.0,1208401.0,6.744880e+08,NaN,4177.0,NaN
1,AC,2005,25059451.9,25712810.3,3.50,18.0,102.0,33504343.0,34377879.0,5.982441e+08,NaN,5122.0,NaN
2,AC,2007,32048964.0,32110602.6,2.00,24.0,96.0,40890361.0,40969004.0,NaN,NaN,NaN,NaN
3,AC,2008,115832532.0,112744065.4,2.00,30.0,126.0,134995354.0,131395945.0,6.819228e+08,NaN,6731.0,NaN
4,AL,2002,51475000.0,52744954.6,2.90,30.0,120.0,106844365.0,109480353.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,TO,2010,693353000.0,692261399.9,1.23,18.0,173.2,698151521.0,697052366.0,NaN,NaN,NaN,0.699
384,TO,2011,267241475.0,269067038.5,2.25,44.8,92.4,256349910.0,258092226.0,4.893865e+09,36154786.9,35292.0,NaN
385,TO,2013,45000000.0,46017626.0,4.88,30.8,56.2,39325614.0,40214920.0,2.860466e+09,183798.7,21368.0,NaN
386,TO,2014,18976000.0,16049428.2,1.76,4.0,72.0,15628697.0,13218362.0,NaN,NaN,NaN,NaN


In [111]:
## Exporting integrated dataset
df.to_csv('bases-ajustadas/operacoes-uf-ano.csv',
          index=False, header=True)

# 5 - Agregando bases externas sem agrupamento

Nesta etapa é feito um dataset integrado com todas as informações.
É importante notar o nível de agregação de cada variável externa na hora de fazer as análises usando este dataset:

- num_empregados = uf/setor/ano/porte_do_cliente
- emissao_co2 = uf/setor/ano
- pib = uf/setor/ano
- idhm = uf/setor

**Importante: Recuperar dataset até o final da operação 3.3 para prosseguir**

In [41]:
## Setting dataset.
df = pd.read_csv('bases-ajustadas/operacoes-ajustado.csv')

In [40]:
## Loading auxiliar datasets.
caged = pd.read_csv('bases-auxiliares/CAGED-ajustado.csv')
pib = pd.read_csv('bases-auxiliares/pib-integrado.csv')
seeg = pd.read_csv('bases-auxiliares/SEEG-emissao-ajustado.csv')
idhm = pd.read_csv('bases-auxiliares/idhm.csv')

In [42]:
## Change column name 'ano' to 'mes'.
df.rename(columns={'ano': 'mes'}, inplace=True)

In [43]:
# Indexing a new column with year format.
df['ano'] = pd.to_datetime(df['data_da_contratacao'])
df['ano'] = pd.DatetimeIndex(df['ano']).year

## 5.1 - CAGED

In [116]:
## Append number of employees from CAGED dataset, segmented by company_size/sector/uf/ano.
df = map_values_caged(df, caged, 'setor_bndes', 'num_empregados')

In [45]:
df.loc[0]

cliente                         ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...
cnpj                                                           61.695.227/0001-93
uf                                                                             SP
numero_do_contrato                                                        2204531
data_da_contratacao                                           2002-02-04T00:00:00
valor_contratado_reais                                                182029685.4
valor_desembolsado_reais                                              182029685.4
fonte_de_recurso_desembolsos                            RECURSOS LIVRES - TESOURO
custo_financeiro                                                        TAXA FIXA
juros                                                                         1.0
prazo_carencia_meses                                                           63
prazo_amortizacao_meses                                                         8
modalidade_de_ap

In [46]:
caged

,ano,uf,setor,porte_do_cliente,num_empregados
0,2019,RO,INDUSTRIA,MICRO,9493
1,2019,AC,INDUSTRIA,MICRO,2372
2,2019,AM,INDUSTRIA,MICRO,8308
3,2019,RR,INDUSTRIA,MICRO,1415
4,2019,PA,INDUSTRIA,MICRO,16603
...,...,...,...,...,...
5827,2002,RS,AGROPECUARIA,GRANDE,2016
5828,2002,MS,AGROPECUARIA,GRANDE,0
5829,2002,MT,AGROPECUARIA,GRANDE,1583
5830,2002,GO,AGROPECUARIA,GRANDE,664


In [39]:
df_fin[(df_fin['uf'] == 'SP') & (df_fin['setor_bndes'] == 'INDUSTRIA')
       & (df_fin['ano'] == '2019') & (df_fin['porte_do_cliente'] == 'GRANDE')]


,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,area_operacional,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato,ano,igpm,valor_contratado_ajustado
13777,LOG & PRINT GRAFICA DADOS VARIAVEIS E LOGISTIC...,66.079.609/0001-06,SP,18206331,2019-01-08T00:00:00,60000000.0,60000000.0,RECURSOS LIVRES - PRÓPRIOS,SELIC,3.9,...,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,OUTRAS,GRANDE,PRIVADA,REAL,ATIVO,2019-01,707.488,38188718.0


In [37]:
df_fin.loc[0]

cliente                         ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...
cnpj                                                           61.695.227/0001-93
uf                                                                             SP
numero_do_contrato                                                        2204531
data_da_contratacao                                           2002-02-04T00:00:00
valor_contratado_reais                                                182029685.4
valor_desembolsado_reais                                              182029685.4
fonte_de_recurso_desembolsos                            RECURSOS LIVRES - TESOURO
custo_financeiro                                                        TAXA FIXA
juros                                                                         1.0
prazo_carencia_meses                                                           63
prazo_amortizacao_meses                                                         8
modalidade_de_ap

In [119]:
search_info(df, 'setor_bndes', 'INDUSTRIA', 'SP', 2019).iloc[0,:]

cliente                         LOG & PRINT GRAFICA DADOS VARIAVEIS E LOGISTIC...
cnpj                                                           66.079.609/0001-06
uf                                                                             SP
numero_do_contrato                                                       18206331
data_da_contratacao                                           2019-01-08T00:00:00
valor_contratado_reais                                                 60000000.0
valor_desembolsado_reais                                               60000000.0
fonte_de_recurso_desembolsos                           RECURSOS LIVRES - PRÓPRIOS
custo_financeiro                                                            SELIC
juros                                                                         3.9
prazo_carencia_meses                                                           24
prazo_amortizacao_meses                                                        36
modalidade_de_ap

## 5.2 - SEEG

In [120]:
## Append SEEG Emissions, segmented by sector/uf/ano.
df = map_values_general(df, seeg, 'setor_bndes', 'emissao_co2')

In [121]:
## Veryfing NA
df['emissao_co2'].isna().sum()

8113

In [122]:
# veryfing random observation.
search_info(seeg, 'setor', 'AGROPECUARIA', 'MG', 2015)


,setor,uf,ano,emissao_co2
718,AGROPECUARIA,MG,2015,87914822.8


In [123]:
search_info(df, 'setor_bndes', 'AGROPECUARIA', 'MG', 2015).iloc[0, :]


cliente                         PLANTAR EMPREENDIMENTOS E PRODUTOS FLORESTAIS ...
cnpj                                                           21.752.910/0001-09
uf                                                                             MG
numero_do_contrato                                                       14213031
data_da_contratacao                                           2015-03-10T00:00:00
valor_contratado_reais                                                 12608000.0
valor_desembolsado_reais                                               7721266.91
fonte_de_recurso_desembolsos                                RECURSOS LIVRES - FAT
custo_financeiro                                                             TJLP
juros                                                                        2.75
prazo_carencia_meses                                                           84
prazo_amortizacao_meses                                                        24
modalidade_de_ap

## 5.3 - PIB

In [124]:
# Append PIB values, segmented by sector/uf/ano.
df = map_values_general(df, pib, 'setor_bndes', 'pib')

In [125]:
## Veryfing NA
df['pib'].isna().sum()


8271

In [126]:
# veryfing random observation.
search_info(pib, 'setor', 'AGROPECUARIA', 'DF', 2010)

,uf,ano,setor,pib
222,DF,2010,AGROPECUARIA,327804575.0


In [127]:
search_info(df, 'setor_bndes', 'AGROPECUARIA', 'DF', 2010).iloc[0, :]

cliente                                                  ASA
cnpj                                      72.600.190/0001-99
uf                                                        DF
numero_do_contrato                                  10201201
data_da_contratacao                      2010-03-25T00:00:00
valor_contratado_reais                            30000000.0
valor_desembolsado_reais                          30000000.0
fonte_de_recurso_desembolsos      RECURSOS LIVRES - PRÓPRIOS
custo_financeiro                                       TJ462
juros                                                   5.54
prazo_carencia_meses                                      12
prazo_amortizacao_meses                                   24
modalidade_de_apoio                             REEMBOLSÁVEL
produto                                          BNDES FINEM
instrumento_financeiro                             PEC BNDES
inovacao                                                 NÃO
area_operacional        

## 5.4 - IDHM

In [128]:
## Append IDH information, segmented by uf/ano.
for i in range(len(idhm)):
    index = np.where(df['ano'].eq(idhm.loc[i, 'ano']) & df['uf'].eq(
        idhm.loc[i, 'uf']))

    df.loc[index[0], 'idhm'] = idhm.loc[i, 'idhm']

In [129]:
## Veryfing NA.
df['idhm'].isna().sum()

11846

In [130]:
## veryfing random observation.
idhm.loc[17:17,:]

,idhm,ano,uf
17,0.674,2010,AM


In [131]:
search_info(df, 'setor_bndes', 'INDUSTRIA', 'AM', 2010)

,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,situacao_do_contrato,mes,igpm,valor_contratado_ajustado,valor_desembolsado_ajustado,ano,num_empregados,emissao_co2,pib,idhm
5690,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,10200971,2010-04-12T00:00:00,40200000.0,40199999.99,RECURSOS LIVRES - FAT / RECURSOS LIVRES - TESOURO,TJLP,2.12,...,LIQUIDADO,2010-04,418.917,43211663.0,43211663.0,2010,61226.0,261072.1,2.149580e+10,0.674
5691,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,10200971,2010-04-12T00:00:00,40200000.0,40199999.99,RECURSOS LIVRES - FAT / RECURSOS LIVRES - TESOURO,TJ462,2.12,...,LIQUIDADO,2010-04,418.917,43211663.0,43211663.0,2010,61226.0,261072.1,2.149580e+10,0.674
5692,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,10200971,2010-04-12T00:00:00,500000.0,500000.00,RECURSOS LIVRES - FAT / RECURSOS LIVRES - TESOURO,TJLP,0.00,...,LIQUIDADO,2010-04,418.917,537458.0,537458.0,2010,61226.0,261072.1,2.149580e+10,0.674
5695,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,10200971,2010-04-12T00:00:00,3200000.0,3200000.00,RECURSOS LIVRES - FAT / RECURSOS LIVRES - TESOURO,TAXA FIXA,4.50,...,LIQUIDADO,2010-04,418.917,3439734.0,3439734.0,2010,61226.0,261072.1,2.149580e+10,0.674


## 5.5 - Número de processos por empresa

In [132]:
fields = ['cnpj', 'processos']
processos = pd.read_csv('bases-auxiliares/bndes_processos.csv', sep=';', usecols=fields)

In [133]:
processos

,cnpj,processos
0,61.695.227/0001-93,105734.0
1,04.229.761/0001-70,548.0
2,04.229.761/0001-70,548.0
3,04.229.761/0001-70,548.0
4,12.282.034/0001-03,2726.0
...,...,...
14950,07.442.711/0001-65,NaN
14951,04.565.361/0001-36,225.0
14952,04.565.361/0001-36,225.0
14953,04.565.361/0001-36,225.0


In [134]:
## Transform processos to a dict with CNPJ
proc_dict = dict(zip(processos['cnpj'], processos['processos']))

In [136]:
## append processos information to our main dataset
df['processos'] = df['cnpj'].map(proc_dict)

In [141]:
## Replacing 0's
df['pib'].replace({'0': np.nan, 0: np.nan}, inplace=True)
df['emissao_co2'].replace({'0': np.nan, 0: np.nan}, inplace=True)
df['num_empregados'].replace({'0': np.nan, 0: np.nan}, inplace=True)
df['idhm'].replace({'0': np.nan, 0: np.nan}, inplace=True)

In [145]:
## Replacing NA to 0 for processos
df['processos'] = df['processos'].replace(np.nan, 0)

In [146]:
## Confirm replace.
df['processos'].isna().sum()

0

In [149]:
# Check for 0's and NA.
df.loc[df['valor_contratado_ajustado'] <= 0]

,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,mes,igpm,valor_contratado_ajustado,valor_desembolsado_ajustado,ano,num_empregados,emissao_co2,pib,idhm,processos


In [152]:
# Check for 0's and NA.
df.loc[df['valor_desembolsado_ajustado'] <= 0]


,cliente,cnpj,uf,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,...,mes,igpm,valor_contratado_ajustado,valor_desembolsado_ajustado,ano,num_empregados,emissao_co2,pib,idhm,processos


In [153]:
# Exporting integrated dataset
df.to_csv('bases-ajustadas/operacoes-integrado.csv',
          index=False, header=True)